In [1]:
!pip3 install pyvespa ir-datasets vespacli
!docker info | grep "Total Memory"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
 Total Memory: 15.62GiB


Listar containers

In [30]:
!docker ps -a

CONTAINER ID   IMAGE               COMMAND                  CREATED       STATUS       PORTS                                                                                      NAMES
b9ee3f3faa2d   vespaengine/vespa   "/usr/local/bin/star…"   2 hours ago   Up 2 hours   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp, 0.0.0.0:19071->19071/tcp, :::19071->19071/tcp   hybridsearch


Para e remove todos containers

In [3]:
!docker stop $(docker ps -aq)
!docker rm $(docker ps -aq)

989dd16a2073
989dd16a2073


In [12]:
import sys
import random
import pickle
import ir_datasets
import re
sys.executable

'/home/codespace/.python/current/bin/python3'

In [20]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    HNSW,
    RankProfile,
    Component,
    Parameter,
    FieldSet,
    GlobalPhaseRanking,
    Function,
)

PACKAGE_NAME = "hybridsearch"
SCHEMA_NAME = "doc"

package = ApplicationPackage(
    name=PACKAGE_NAME,
    schema=[
        Schema(
            name=SCHEMA_NAME,
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(
                        name="body",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                        bolding=True,
                    ),
                    Field(
                        name="embedding",
                        type="tensor(x[384])",
                        indexing=[
                            'input body',
                            "embed",
                            "index",
                            "attribute",
                        ],
                        ann=HNSW(distance_metric="angular"),
                        is_document_field=False,
                    ),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["body"])],
            rank_profiles=[
                RankProfile(
                    name="bm25",
                    inputs=[("query(q)", "tensor(x[384])")],
                    functions=[
                        Function(name="bm25sum", expression="bm25(body)")
                    ],
                    first_phase="bm25sum",
                ),
                RankProfile(
                    name="semantic",
                    inputs=[("query(q)", "tensor(x[384])")],
                    first_phase="closeness(field, embedding)",
                ),
                RankProfile(
                    name="fusion",
                    inherits="bm25",
                    inputs=[("query(q)", "tensor(x[384])")],
                    first_phase="closeness(field, embedding)",
                    global_phase=GlobalPhaseRanking(
                        expression="reciprocal_rank_fusion(bm25sum, closeness(field, embedding))",
                        rerank_count=1000,
                    ),
                ),
            ],
        )
    ],
    components=[
        Component(
            id="e5",
            type="hugging-face-embedder",
            parameters=[
                Parameter(
                    "transformer-model",
                    {
                        "url": "https://github.com/vespa-engine/sample-apps/raw/master/examples/model-exporting/model/e5-small-v2-int8.onnx"
                    },
                ),
                Parameter(
                    "tokenizer-model",
                    {
                        "url": "https://raw.githubusercontent.com/vespa-engine/sample-apps/master/examples/model-exporting/model/tokenizer.json"
                    },
                ),
            ],
        )
    ],
)

In [7]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=package)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for configuration server, 10/60 seconds...
Waiting for configuration server, 15/60 seconds...
Waiting for configuration server, 20/60 seconds...
Waiting for configuration server, 25/60 seconds...
Waiting for configuration server, 30/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Waiting for application to come up, 15/300 seconds.
Waiting for application to come up, 20/300 seconds.
Waiting for application to come up, 25/300 seconds.
Waiting for application to come up, 30/300 seconds.
Waiting for application to come up, 35/300 seconds.
Waiting for application to come up, 40/300 seconds.
Waiting for application to come up, 45/300 seconds.
Waiting for application to come up, 50/300 seconds.
Waiting for application to come up, 55/300 seconds.
Waiting for application

In [13]:
random.seed(42)

def load_dataset(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)

data_set = "subset_msmarco_train_0/subset_msmarco_train_0.01_99.pkl"

data = load_dataset(data_set)
queries = data["queries"]
documents = data["docs"]

# Split the queries (queries is a dictionary of {query_id: query_object})
query_ids = list(queries.keys())  # List of query IDs

# Shuffle query IDs to ensure a random split
random.shuffle(query_ids)

# Split into 80% for training, 20% for validation
split_ratio = 0.8
train_query_ids = query_ids[:int(len(query_ids) * split_ratio)]
test_query_ids = query_ids[int(len(query_ids) * split_ratio):]

train_queries = {qid: queries[qid] for qid in train_query_ids}
test_queries = {qid: queries[qid] for qid in test_query_ids}

In [14]:
def remove_control_characters(text):
    # Remove control characters using regex
    return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

In [27]:
vespa_cli_feed = [
    {
        "put": f"id:{PACKAGE_NAME}:{SCHEMA_NAME}::{doc.doc_id}",
        "fields": {
            "body": remove_control_characters(doc.text),
            "id": doc.doc_id,
        }
    }
    for doc in documents.values()
]

# write to jsonl file
import json
import os

def write_jsonl(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')
os.makedirs("data", exist_ok=True)
write_jsonl(vespa_cli_feed, "data/vespa_cli_feed.jsonl")

In [ ]:
output_list = !vespa feed data/vespa_cli_feed.jsonl
results = json.loads("".join(output_list))
print(results)

In [ ]:
# vespa_feed = [
#     {"id": doc.doc_id, "fields": {"body": remove_control_characters(doc.text), "id": doc.doc_id}}
#     for doc in documents.values()
# ]

In [ ]:
# use only 2% of the documents for the demo
# vespa_feed = vespa_feed[:int(len(vespa_feed) * 0.02)]

In [3]:
from vespa.io import VespaResponse, VespaQueryResponse


# def callback(response: VespaResponse, id: str):
#     if not response.is_successful():
#         print(f"Error when feeding document {id}: {response.get_json()}")


# app.feed_iterable(vespa_feed, schema="doc", namespace="tutorial", callback=callback)

In [4]:
import pandas as pd


def display_hits_as_df(response: VespaQueryResponse, fields) -> pd.DataFrame:
    records = []
    for hit in response.hits:
        record = {}
        for field in fields:
            record[field] = hit["fields"][field]
        records.append(record)
    return pd.DataFrame(records)

In [33]:
from vespa.application import Vespa

app = Vespa(url="http://localhost:8080")

In [9]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() limit 5",
        query=query,
        ranking="bm25",
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "body"]))

                             id  \
0  msmarco_passage_03_503441468   
1  msmarco_passage_00_465189549   
2  msmarco_passage_04_521200350   
3  msmarco_passage_00_769101016   
4  msmarco_passage_01_512229249   

                                                body  
0  Shopping for Raw Foods. <hi>Fruits</hi> <hi>an...  
1  Substitute processed carbohydrates with large ...  
2  <hi>How</hi> to Broil All Types of Foods for F...  
3  <hi>Fruits</hi> <hi>and</hi> <hi>Vegetables</h...  
4  On these pages, you will learn the properties ...  


In [26]:
from vespa.evaluation import VespaEvaluator

test_queries_dict = {
    q.query_id: q.text
    for q in test_queries.values()
}

relevant_docs = dict()
for qrel in data["qrels"]:
    relevant_docs[qrel.query_id] = relevant_docs.get(qrel.query_id, set())
    relevant_docs[qrel.query_id].add(qrel.doc_id)

def query_fn_for_ranking(ranking: str) -> callable:
    def query_fn(query_text: str, top_k: int) -> dict:
        return {
            "yql": f"select * from sources * where userQuery()",
            "query": query_text,
            "hits": top_k,
            "ranking": ranking,
        }
    return query_fn

for ranking in ["bm25", "semantic", "fusion"]:
    query_fn = query_fn_for_ranking(ranking)
    evaluator = VespaEvaluator(
        queries=test_queries_dict,
        relevant_docs=relevant_docs,
        vespa_query_fn=query_fn,
        app=app,
        name=f"test-run-{ranking}",
        accuracy_at_k=[10],
        precision_recall_at_k=[100],
        mrr_at_k=[10],
        ndcg_at_k=[10],
        write_csv=True
    )
    
    results = evaluator.run()
    print(f"Results for {ranking}:")
    print("Primary metric:", evaluator.primary_metric)
    print("All results:", results)

RetryError: RetryError[<Future at 0x74cb27f98c50 state=finished raised ReadTimeout>]

In [16]:
print(query_fn_for_ranking("bm25")("How Fruits and Vegetables Can Treat Asthma?", 5))

{'yql': 'select * from sources * where userQuery()', 'query': 'How Fruits and Vegetables Can Treat Asthma?', 'hits': 5, 'ranking': 'bm25'}


In [35]:
app.get_data(SCHEMA_NAME, vespa_cli_feed[1]["fields"]["id"]).json

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /document/v1/doc/doc/docid/msmarco_passage_31_211784428 (Caused by ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')))

In [32]:
app.get_data(SCHEMA_NAME, vespa_cli_feed[-1]["fields"]["id"]).json

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /document/v1/doc/doc/docid/msmarco_passage_02_869363661 (Caused by ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')))

Remover container

In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()